In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from pandas import DataFrame

Property prices

In [ ]:
a=requests.get("https://www.makaan.com/price-trends/property-rates-for-buy-in-mumbai").text
soup=bs(a,'lxml')

In [ ]:
titles=[] ## title
pricemin=[] ## min price
pricemax=[] ##max price
priceavg=[] ##avg price
pricerise=[] ##price rise

In [ ]:
tb=soup.select("table[data-trend-type='apartment'] > tbody > tr")
for td in tb:
	try:
		title=td.select("td[data-source='locality']")[0].get_text()
		p1=td.select("td")[1].select("[itemprop='minPrice']")[0].get_text().replace(",","")
		p2=td.select("td")[1].select("[itemprop='maxPrice']")[0].get_text().replace("/ sqft","").replace(",","")
		pavg=td.select("td")[2].get_text().replace("/ sqft","").replace(",","")
		prise=td.select("td")[3].get_text().replace("%","").replace(",","")
		titles.append(title)
		pricemin.append(p1)
		pricemax.append(p2)
		priceavg.append(pavg)
		pricerise.append(prise)
	except:
		pass



In [ ]:
df=DataFrame({"Localty Name":titles,
						"Price min":pricemin,
						"Price Max":pricemax,
						"Price AVG":priceavg,
						"Price Rise":pricerise})

In [ ]:
print(df)

        Localty Name Price min Price Max    Price AVG Price Rise
0         Thane West         0   194951    101187.33        19.8
1     Mira Road East       678   197266     92444.61         7.2
2           Kharghar       648   109948     56700.82         -19
3            Chembur      1240    48273     39984.16        31.3
4     Kandivali East       219   172571     68358.17        13.4
5              Powai       750    85308     51426.88       -12.7
6      Goregaon East       714    60771     36444.47        48.6
7          Dombivali       550    17094     14877.86       -32.3
8               Ulwe       707   225193     61486.98        44.4
9       Andheri East      1167   114444        54111       -16.1
10             Virar       347    76212     11771.18         5.9
11            Panvel      2200    77778     27260.07        32.1
12      Andheri West      1750   118750     61324.84          43
13       Mulund West      3304    32553     29609.91        50.5
14        Malad West     

In [ ]:
df.head()

,Localty Name,Price min,Price Max,Price AVG,Price Rise
0,Thane West,0,194951,101187.33,19.8
1,Mira Road East,678,197266,92444.61,7.2
2,Kharghar,648,109948,56700.82,-19
3,Chembur,1240,48273,39984.16,31.3
4,Kandivali East,219,172571,68358.17,13.4


In [ ]:
newcols = {
    'Localty Name': 'Localty', 
    'Price min': 'Min', 
    'Price Max': 'Max',
    'Price AVG': 'Avg',
    'Price Rise':'Change'
}

In [ ]:
df.rename(columns=newcols, inplace=True)

In [ ]:
df.head()

,Localty,Min,Max,Avg,Change
0,Thane West,0,194951,101187.33,19.8
1,Mira Road East,678,197266,92444.61,7.2
2,Kharghar,648,109948,56700.82,-19
3,Chembur,1240,48273,39984.16,31.3
4,Kandivali East,219,172571,68358.17,13.4


In [18]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
#df.to_csv('/content/drive/MyDrive/IBM/MumbaiHousePrices.csv')

In [ ]:
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

In [ ]:
locator = Nominatim(user_agent='myGeocoder')


In [ ]:
from geopy.extra.rate_limiter import RateLimiter

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df['location'] = df['Localty'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

In [16]:
dfx=df.dropna()

In [17]:
dfx.head()

,Localty,Min,Max,Avg,Change,location,point,latitude,longitude,altitude
0,Thane West,0,194951,101187.33,19.8,"(Thane, Maharashtra, India, (19.1943294, 72.97...","(19.1943294, 72.9701779, 0.0)",19.194329,72.970178,0.0
1,Mira Road East,678,197266,92444.61,7.2,"(Mira Road, Naya Nagar Road, Mira Road, Thane,...","(19.2816975, 72.8560676, 0.0)",19.281697,72.856068,0.0
2,Kharghar,648,109948,56700.82,-19,"(Kharghar, Panvel, Raigad, Maharashtra, 410206...","(19.0525298, 73.0735111, 0.0)",19.052530,73.073511,0.0
3,Chembur,1240,48273,39984.16,31.3,"(Chembur, Prakash Thorat Marg, Phule Nagar, M/...","(19.0626319, 72.9011399, 0.0)",19.062632,72.901140,0.0
4,Kandivali East,219,172571,68358.17,13.4,"(Kandivali East, R/S Ward, Zone 4, Mumbai, Mum...","(19.2103809, 72.8640837, 0.0)",19.210381,72.864084,0.0


In [19]:
dfx.to_csv('/content/drive/MyDrive/IBM/MumbaiHousePrices_Geoloc.csv')

Remove None 

In [ ]:
url='https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Mumbai'
source = requests.get(url).text
soup=BeautifulSoup(source,'lxml') #parse in the soup

tab=str(soup.table)
dfs=pd.read_html(tab)
df=dfs[0]

df=df[df['Location'].str.contains(",")].reset_index(drop=True)


new=df["Location"].str.split(",",n=1,expand=True) #location acts as the basis of 2 new columns
df["Zone"]= new[0]
df["Neighbourhood"]= new[1]


print(df)


                                            Area  ...    Neighbourhood
0                                         Amboli  ...  Western Suburbs
1                                     D.N. Nagar  ...  Western Suburbs
2                                 Four Bungalows  ...  Western Suburbs
3                                    Lokhandwala  ...  Western Suburbs
4                                          Marol  ...  Western Suburbs
5                                          Sahar  ...  Western Suburbs
6                                Seven Bungalows  ...  Western Suburbs
7                                        Versova  ...  Western Suburbs
8                                      Mira Road  ...  Western Suburbs
9                                      Bhayandar  ...  Western Suburbs
10                                         Uttan  ...  Western Suburbs
11                           Bandstand Promenade  ...  Western Suburbs
12                                      Kherwadi  ...  Western Suburbs
13    

In [ ]:
df.head()

,Area,Location,Latitude,Longitude,Zone,Neighbourhood
0,Amboli,"Andheri,Western Suburbs",19.129300,72.843400,Andheri,Western Suburbs
1,D.N. Nagar,"Andheri,Western Suburbs",19.124085,72.831373,Andheri,Western Suburbs
2,Four Bungalows,"Andheri,Western Suburbs",19.124714,72.827210,Andheri,Western Suburbs
3,Lokhandwala,"Andheri,Western Suburbs",19.130815,72.829270,Andheri,Western Suburbs
4,Marol,"Andheri,Western Suburbs",19.119219,72.882743,Andheri,Western Suburbs
